# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.3 
Current idle_timeout is 2800 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::462533425464:role/susheel-glue-notebook
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 1b3f5b61-0a2d-45fd-94a2-77cf23bfff61
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.3
--enable-glue-datacatalog true
Waiting for session 1b3f5b61-

In [2]:
import requests
import json
import pandas as pd  

# Set the API endpoint and headers
api_url = "https://hexaware.centralindia.cloudapp.azure.com/Profisee/rest/v1/Records/96035_CustomerAccount?PageSize=350" # Replace with the actual API endpoint
# Replace with your Profisee API key

header = {
"accept": "application/json",
"x-api-key": "3ed16c4888cb4469a4d7a1e9d26a24d4",
"Content-Type": "application/json-patch+json" 
}


# Prepare the payload data

# Send the get request for pushing new data 

response = requests.get(api_url,headers=header)
# print(response)
jsonResponse = response.json()
# print("Entire JSON response")
# print(jsonResponse)
mydf = jsonResponse["data"]
print(mydf)


# Check the response
if response.status_code == 200:
# Request was successful
    response_data = response.json()
    print(response_data)
else:
# Request failed
    print({response.status_code})

[{'internalId': 1761, 'id': '54de5014-cd43-4e4b-b8d4-527d33cb631c', 'code': '150', 'name': 'Allround Sports, Inc.', 'companyType': 'RET', 'accountGroup': 'CG16', 'status': 'A', 'segment': 'TIER2', 'sourceSystem': 'JDE', 'sourceCode': '150', 'zendeskAccountId': '', 'jdEdwardAccountId': '150', 'businessNumber': '48170582759', 'sales': '543', 'addressLine1': '12 WEAVING COURT', 'addressLine2': '', 'addressLine3': '', 'city': 'ALICE SPRINGS', 'stateProvince': 'NT', 'postalCode': '7001', 'country': 'AU', 'webSite': 'www.allroundsports.com.au', 'phoneNumber': '', 'emailAddress': 'rebbecca.didio@allroundsports.com.au', 'std Full Address': '12 Weaving Ct, Araluen, Northern Territory 0870, Australia', 'std Address': '12 Weaving Ct', 'std City': 'Alice Springs', 'std State': 'Northern Territory', 'std ZIP': '0870', 'std Country': 'Australia', 'std Country Code - 2 Char': 'AU', 'std County': 'Alice Springs', 'latitude': -23.7024562, 'longitude': 133.8504422, 'location Confidence': '4', 'location 

In [3]:
import json
import boto3

s3 = boto3.client('s3')
json_object = mydf
s3.put_object(
     Body=json.dumps(json_object),
     Bucket='susheel-poc-glue',
     Key='Boto/Data.json'
)

{'ResponseMetadata': {'RequestId': 'J4XRQ6DGASF691HG', 'HostId': 'RqhNZJB17QoGIwcvMpaq8NtF0j+AKl4mYlRukVnlRas8zg47wjvv31iv4K10Tn2r/RDArD/jFsw=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'RqhNZJB17QoGIwcvMpaq8NtF0j+AKl4mYlRukVnlRas8zg47wjvv31iv4K10Tn2r/RDArD/jFsw=', 'x-amz-request-id': 'J4XRQ6DGASF691HG', 'date': 'Wed, 14 Jun 2023 05:38:57 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"fcdf4292a13e0e23ee2187e07e19632d"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'ETag': '"fcdf4292a13e0e23ee2187e07e19632d"', 'ServerSideEncryption': 'AES256'}


In [5]:
from pyspark.sql import SparkSession
my_spark = SparkSession.builder.appName('Practice').getOrCreate()
boto_read = my_spark.read.format("json").load("s3://susheel-poc-glue/Boto/Data.json")

In [6]:
boto_read.count()

350


In [7]:
boto_read.show()

+-----------------------------------------------+------------+--------------------+------------+-------------+--------------+--------------+---------+---------+---------------+----+-----------+---------+--------------------+--------------------+--------------------+----------------+--------------------+----------+-----------------+--------------------+--------------------+-----------+-------------------+-----------------------+----------------------------+-------------+-------------+--------------------+------------+----------------+------------+--------------------+----------+----------------+--------------------+--------------+----------+--------------+-----+-------+----------+------------+-------------+------+--------------------+-------------+-------------+-------------------------+----------------+--------------------+------------------+-------+------------------+--------------------+----------------+
|96035_AddressVerificationCA Verification Status|accountGroup|        addressLin

In [8]:
boto_read.printSchema()

root
 |-- 96035_AddressVerificationCA Verification Status: string (nullable = true)
 |-- accountGroup: string (nullable = true)
 |-- addressLine1: string (nullable = true)
 |-- addressLine2: string (nullable = true)
 |-- addressLine3: string (nullable = true)
 |-- approved Count: string (nullable = true)
 |-- businessNumber: string (nullable = true)
 |-- canDelete: boolean (nullable = true)
 |-- canUpdate: boolean (nullable = true)
 |-- city: string (nullable = true)
 |-- code: string (nullable = true)
 |-- companyType: string (nullable = true)
 |-- country: string (nullable = true)
 |-- emailAddress: string (nullable = true)
 |-- enterDTM: string (nullable = true)
 |-- enterUserName: string (nullable = true)
 |-- golden Record: string (nullable = true)
 |-- id: string (nullable = true)
 |-- internalId: long (nullable = true)
 |-- jdEdwardAccountId: string (nullable = true)
 |-- lastChgDTM: string (nullable = true)
 |-- lastChgUserName: string (nullable = true)
 |-- latitude: double (n

In [9]:
boto_read.write.format("csv").option('header',True).option('inferSchema',True).mode("overwrite").option("path","s3://susheel-poc-glue/test/").save()

In [11]:
csv_read = my_spark.read.format("csv").option('header',True).option('inferSchema',True).load("s3://susheel-poc-glue/test/*.csv")

In [12]:
csv_read.show()

+-----------------------------------------------+------------+--------------------+------------+-------------+--------------+--------------+---------+---------+---------------+----+-----------+---------+--------------------+--------------------+--------------------+----------------+--------------------+----------+-----------------+--------------------+--------------------+-----------+-------------------+-----------------------+----------------------------+-------------+-------------+--------------------+------------+----------------+------------+--------------------+----------+----------------+--------------------+--------------+----------+--------------+-----+-------+----------+------------+-------------+------+--------------------+-------------+-------------+-------------------------+----------------+--------------------+------------------+-------+------------------+--------------------+----------------+
|96035_AddressVerificationCA Verification Status|accountGroup|        addressLin

In [13]:
csv_read.printSchema()

root
 |-- 96035_AddressVerificationCA Verification Status: integer (nullable = true)
 |-- accountGroup: string (nullable = true)
 |-- addressLine1: string (nullable = true)
 |-- addressLine2: string (nullable = true)
 |-- addressLine3: string (nullable = true)
 |-- approved Count: string (nullable = true)
 |-- businessNumber: string (nullable = true)
 |-- canDelete: boolean (nullable = true)
 |-- canUpdate: boolean (nullable = true)
 |-- city: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- companyType: string (nullable = true)
 |-- country: string (nullable = true)
 |-- emailAddress: string (nullable = true)
 |-- enterDTM: timestamp (nullable = true)
 |-- enterUserName: string (nullable = true)
 |-- golden Record: string (nullable = true)
 |-- id: string (nullable = true)
 |-- internalId: integer (nullable = true)
 |-- jdEdwardAccountId: integer (nullable = true)
 |-- lastChgDTM: timestamp (nullable = true)
 |-- lastChgUserName: string (nullable = true)
 |-- latitud

In [14]:
NewColumns=(column.replace(' ', '_') for column in csv_read.columns)
csv_read = csv_read.toDF(*NewColumns)

In [15]:
new_column_name_list= list(map(lambda x: x.lower(), csv_read.columns))
csv_read = csv_read.toDF(*new_column_name_list)

In [16]:
csv_read.printSchema()

root
 |-- 96035_addressverificationca_verification_status: integer (nullable = true)
 |-- accountgroup: string (nullable = true)
 |-- addressline1: string (nullable = true)
 |-- addressline2: string (nullable = true)
 |-- addressline3: string (nullable = true)
 |-- approved_count: string (nullable = true)
 |-- businessnumber: string (nullable = true)
 |-- candelete: boolean (nullable = true)
 |-- canupdate: boolean (nullable = true)
 |-- city: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- companytype: string (nullable = true)
 |-- country: string (nullable = true)
 |-- emailaddress: string (nullable = true)
 |-- enterdtm: timestamp (nullable = true)
 |-- enterusername: string (nullable = true)
 |-- golden_record: string (nullable = true)
 |-- id: string (nullable = true)
 |-- internalid: integer (nullable = true)
 |-- jdedwardaccountid: integer (nullable = true)
 |-- lastchgdtm: timestamp (nullable = true)
 |-- lastchgusername: string (nullable = true)
 |-- latitud

In [17]:
csv_read = csv_read.withColumnRenamed("96035_addressverificationca_verification_status","_96035_addressverificationca_verification_status").withColumnRenamed("std_country_code_-_2_char","std_country_code")

In [18]:
csv_read.printSchema()

root
 |-- _96035_addressverificationca_verification_status: integer (nullable = true)
 |-- accountgroup: string (nullable = true)
 |-- addressline1: string (nullable = true)
 |-- addressline2: string (nullable = true)
 |-- addressline3: string (nullable = true)
 |-- approved_count: string (nullable = true)
 |-- businessnumber: string (nullable = true)
 |-- candelete: boolean (nullable = true)
 |-- canupdate: boolean (nullable = true)
 |-- city: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- companytype: string (nullable = true)
 |-- country: string (nullable = true)
 |-- emailaddress: string (nullable = true)
 |-- enterdtm: timestamp (nullable = true)
 |-- enterusername: string (nullable = true)
 |-- golden_record: string (nullable = true)
 |-- id: string (nullable = true)
 |-- internalid: integer (nullable = true)
 |-- jdedwardaccountid: integer (nullable = true)
 |-- lastchgdtm: timestamp (nullable = true)
 |-- lastchgusername: string (nullable = true)
 |-- latitu

In [19]:
csv_read.count()

350


In [20]:
csv_read.write.format("csv").option('header',True).option('inferSchema',True).mode("overwrite").option("path","s3://susheel-poc-glue/transform/").save()